$$ D_{KL}(p||q) ≈ D_{KL}(\hat{p}||\hat{q}) + \frac{1}{2}(p - \hat{p})^T I(\hat{p}) (p - \hat{p}) $$


In [28]:
import numpy as np
from scipy.stats import entropy

import numpy as np

def KL_divergence(p, q):
    p = np.asarray(p)
    q = np.asarray(q)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

p = np.array([0.3,0.7])
q = np.array([0.2, 0.8])

print("KL divergence: ", KL_divergence(p, q))



def fisher_information_matrix(p, q):
    return np.matrix(np.diag(np.array([p[i] * q[i] / (p[i] - q[i]) ** 2 for i in range(len(p))])))



print("Fisher Information Matrix: \n", fisher_information_matrix(p, q))


KL divergence:  0.02816755759528336
Fisher Information Matrix: 
 [[ 6.  0.]
 [ 0. 56.]]


In [29]:
-1/2 * ((p-q) @ fisher_information_matrix(p, q).T @ (p-q))

matrix([[-0.31]])